In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os
import sys

In [2]:
sc = SparkContext()
spark = SparkSession(sc)

In [3]:
parse_results_remote_dir = os.path.join('D:\\bio-ner\\tsv', 'pubmed_data', 'tsv_files')

In [4]:
timestart = datetime.datetime.now()
num_xml_files = 2 
batch_size =50
pubmed_tsv_file = os.path.join(parse_results_remote_dir, 'batch#{}.tsv'.format(1))   
print("Reading file {}".format(pubmed_tsv_file))     
abstracts_batch_df = spark.read.csv(path=pubmed_tsv_file, header=True, inferSchema=True, sep = "\t")

print("\tAdding {} records ...".format(abstracts_batch_df.count()))
abstracts_full_df = abstracts_batch_df

Reading file D:\bio-ner\tsv\pubmed_data\tsv_files\batch#1.tsv
	Adding 26828 records ...


In [5]:
for i in range(1 + batch_size, num_xml_files + 1, batch_size):  
    try:
        pubmed_tsv_file = os.path.join(parse_results_remote_dir, 'batch#{}.tsv'.format(i))   
        print("Reading file {}".format(pubmed_tsv_file))     
        abstracts_batch_df = spark.read.csv(path=pubmed_tsv_file, header=True, inferSchema=True, sep = "\t")

        print("\tAdding {} records ...".format(abstracts_batch_df.count()))
        
        abstracts_full_df = abstracts_full_df.union(abstracts_batch_df)
       
    except:
        print("Skipped" + str(i))

In [6]:
abstracts_full_df.printSchema()
print("abstracts_full_df.count() = {}".format(abstracts_full_df.count()))
print("abstracts_full_df.head() = {}".format(abstracts_full_df.head()))

root
 |-- pmid: integer (nullable = true)
 |-- abstract: string (nullable = true)

abstracts_full_df.count() = 26828
abstracts_full_df.head() = Row(pmid=30978, abstract='The paper describes a modified method of isolating the branching enzyme of amylose isomerase from muscles and a study of the enzyme activity at different stages of purification. By enzyme fractionation on biogel R-150 and Sepharose 6B the fractions containing different RNA amounts have been isolated. The activity of fractions has been shown to depend on their content of RNA. The paper presents a procedure used to isolate a highly purified fraction of amylose isomerase and its properties (pH and temperature optima, enzyme optimal concentration and Michaelis constant).')


In [7]:
timeend = datetime.datetime.now()
timedelta = round((timeend - timestart).total_seconds() / 60, 2)
print("Time taken to execute above cell: " + str(timedelta) + " mins")

abstracts_full_df2 = abstracts_full_df

Time taken to execute above cell: 0.48 mins


In [8]:
from pyspark.sql.functions import regexp_replace, trim, col, lower, udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover

In [9]:
print("abstracts_full_df2.head() = {}".format(abstracts_full_df2.head()))

# Convert the content to Lower Case
print("Converting the abstarct to Lower Case ... ")
abstracts_full_df3 = abstracts_full_df2.withColumn("abstractNew", lower(col("abstract"))).\
    withColumn("abstractNew", regexp_replace("abstractNew", '[^\w-_ ]', ""))

abstracts_full_df3.printSchema()
print("abstracts_full_df3.head() = {}".format(abstracts_full_df3.head()))

# Tokenize the Abstracts
print("tokenizating the abstracts... ")
tokenizer = Tokenizer(inputCol="abstractNew", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtWords")

abstracts_full_df4 = tokenizer.transform(abstracts_full_df3)

print("After tokenization: ")
abstracts_full_df4.printSchema()
print("abstracts_full_df4.count() = {}".format(abstracts_full_df4.count()))
print("abstracts_full_df4.head() = {}".format(abstracts_full_df4.head()))

# PRINT HOW MUCH TIME IT TOOK TO RUN THE CELL
timeend = datetime.datetime.now()
timedelta = round((timeend - timestart).total_seconds() / 60, 2)
print("Time taken to execute above cell: " + str(timedelta) + " mins")

abstracts_full_df2.head() = Row(pmid=30978, abstract='The paper describes a modified method of isolating the branching enzyme of amylose isomerase from muscles and a study of the enzyme activity at different stages of purification. By enzyme fractionation on biogel R-150 and Sepharose 6B the fractions containing different RNA amounts have been isolated. The activity of fractions has been shown to depend on their content of RNA. The paper presents a procedure used to isolate a highly purified fraction of amylose isomerase and its properties (pH and temperature optima, enzyme optimal concentration and Michaelis constant).')
Converting the abstarct to Lower Case ... 
root
 |-- pmid: integer (nullable = true)
 |-- abstract: string (nullable = true)
 |-- abstractNew: string (nullable = true)

abstracts_full_df3.head() = Row(pmid=30978, abstract='The paper describes a modified method of isolating the branching enzyme of amylose isomerase from muscles and a study of the enzyme activity at dif

In [10]:
timestart = datetime.datetime.now()
model = None
window_size = 5
vector_size = 50
min_count =1000
print("Start training the model ...")
word2Vec = Word2Vec(windowSize = window_size, vectorSize = vector_size, minCount=min_count, numPartitions=10, inputCol="words", outputCol="result")
model = word2Vec.fit(abstracts_full_df4)

Start training the model ...


In [11]:
timeend = datetime.datetime.now()
timedelta = round((timeend - timestart).total_seconds() / 60, 2)
print("model.getVectors().count() = {}".format(model.getVectors().count()))
print("model.getVectors().head() = {}".format(model.getVectors().head()))
print("Time taken to train the word2Vec model: " + str(timedelta) + " mins")

model.getVectors().count() = 442
model.getVectors().head() = Row(word='rate', vector=DenseVector([-0.9451, -1.7659, -0.1507, 3.5502, -0.7239, 0.4562, -0.253, -2.8837, 2.1748, 4.4777, -0.3117, -1.3058, 5.2181, -0.8886, -0.0424, -0.1025, -1.1013, 4.0176, 0.2005, 1.1202, 0.5815, -1.9429, 1.54, 1.0596, -5.89, 2.5988, 2.9339, -0.2831, 1.8331, -1.7408, 4.1616, 0.1448, 0.1951, 0.0499, -0.3289, -4.6675, 3.1592, 0.5372, -0.5875, -2.3299, -1.1386, 1.1978, 2.7699, -1.9725, -4.5962, -3.1193, 1.0886, -1.9481, 5.1783, 4.276]))
Time taken to train the word2Vec model: 0.38 mins


In [13]:
print("findSynonyms('heart') = {}".format(model.findSynonyms("heart", 20).select("word").head(20)))

findSynonyms('heart') = [Row(word='fetal'), Row(word='flow'), Row(word='pressure'), Row(word='arterial'), Row(word='blood'), Row(word='rate'), Row(word='oxygen'), Row(word='decreased'), Row(word='renal'), Row(word='fluid'), Row(word='increased'), Row(word='urine'), Row(word='mean'), Row(word='liver'), Row(word='plasma'), Row(word='muscle'), Row(word='significantly'), Row(word='brain'), Row(word='subjects'), Row(word='rates')]


In [24]:
print(abstracts_full_df4.count())
print(model.getVectors().count())
model.getVectors().printSchema()

26828
442
root
 |-- word: string (nullable = true)
 |-- vector: vector (nullable = true)



In [25]:
"a b " * 10

'a b a b a b a b a b a b a b a b a b a b '